In [1]:
# RePORTER data

[RePORTER data](https://reporter.nih.gov/exporter/): publications + funding data of NIH projects (US) from 1980 until 2022.

Analysis time window: `1985 - 2022`.

## Imports

In [2]:
import os
import json
import math
import zipfile
import pandas as pd

## Scraper

[pynih](https://github.com/jermwatt/pynih) is a useful library designed to extract data from the [NIH RePORTER API](https://api.reporter.nih.gov/). However, it was not utilized for the Hackathon, where zip files were manually downloaded.

## Functions

In [3]:
def create_folder(folder_path):
    # Specify the folder containing the zip files
    # Create the extraction folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

In [4]:
def create_general_folders():
    folders = []
    for folder in ['raw', 'input', 'processing', 'output']:
        folder_path = os.path.join('data/', folder)
        folders.append(create_folder(folder_path))
    return folders

In [5]:
def create_extraction_folders(document):
    # Specify the folder containing the zip files
    zip_folder = f'/{document}/'
    # Specify the folder where you want to extract the files
    extracted_folder = f'data/input/{document}/'
    # Create the extraction folder if it doesn't exist
    os.makedirs(zip_folder, exist_ok=True)
    os.makedirs(extracted_folder, exist_ok=True)
    print(f'Input folder: {zip_folder}')
    print(f'Output folder: {zip_folder}')
    return zip_folder, extracted_folder

In [6]:
def unzip_files(zip_folder, extracted_folder):
    # List all zip files in the zip folder
    zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]

    # Loop through zip files and extract contents
    count = 0
    for zip_file in zip_files:
        zip_path = os.path.join(zip_folder, zip_file)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_folder)
        count += 1
    print(f'{count} zips extracted successfully!')

In [7]:
def check_list_integrity(my_list):
    # Check for None values
    contains_none = any(x is None for x in my_list)
    print(f'None: {contains_none}')
    # Check for NaN values
    contains_nan = any(pd.isna(x) for x in my_list)
    print(f'Nan: {contains_nan}')
    # Check for null values (assuming you have imported the math module)
    contains_null = any(math.isnan(x) for x in my_list if isinstance(x, (int, float)))
    print(f'Null: {contains_null}')

In [8]:
def count_duplicates(df):
    # Check for duplicates based on all columns
    duplicates = df[df.duplicated()]
    # Print the duplicate rows
    print(f"Duplicate Rows: {duplicates.shape[0]}")

In [9]:
def count_repeat_values(df, col_name):
    # Use value_counts to count occurrences of each unique value in the column
    value_counts = df[col_name].value_counts()

    # Find values that repeat multiple times (more than once, adjust as needed)
    repeated_values = value_counts[value_counts > 1].index.tolist()

    # Print the values that repeat multiple times
    print(f"Values that repeat multiple times: {len(repeated_values)}")
    return repeated_values

In [10]:
def create_df_from_csv(folders, document, projects_number=''):
    extracted_folder =  os.path.join(folders[1], document)
    # Create a DataFrame to store the data from the CSVs
    df_list = []
    # Loop through extracted CSV files and read them into the DataFrame
    for csv_file in os.listdir(extracted_folder):
        if csv_file.endswith('.csv'):
            csv_path = os.path.join(extracted_folder, csv_file)
            try:
                # Open and read the file manually with error handling
                with open(csv_path, 'r', encoding='utf-8', errors='replace') as file:
                    df = pd.read_csv(file, on_bad_lines='skip', encoding='utf-8', engine='python')
                    df.columns = df.columns.str.lower()
                    if 'projects' in csv_path and projects_number != '':
                        # print(df.columns)
                        df = df[df["core_project_num"].isin(projects_number)]
                        # df = pd.merge(codes_df, df, on='pmid', how='left')
                df_list.append(df)
            except Exception as e:
                print(f"Error reading {csv_path}: {e}")

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(df_list, ignore_index=True)
    print(f'Number of {document}: {final_df.shape[0]}')
    return final_df


## Paths

In [11]:
folders = create_general_folders() # 'raw', 'input', 'processing', 'output'
folders

['data/raw', 'data/input', 'data/processing', 'data/output']

## Unzip files

In [12]:
zip_folder = os.path.join(folders[0], 'publications')
extracted_folder = os.path.join(folders[1], 'publications')

In [13]:
# UNZIP FILES - optional
# if any(os.scandir(zip_folder)):
#    unzip_files(zip_folder, extracted_folder)
# else:
#    print('No zips!')

## Data

### Count Data

In [14]:
json_file_path = 'data/results_for_analysis.json'

In [15]:
# Open the JSON file
with open(json_file_path, 'r') as file:
    # Load the JSON data
    data = json.load(file)

In [16]:
count_data = pd.DataFrame.from_dict(data)
count_data.head(2)

,pmcid,agg_sentence_index,agg_n_fem,agg_n_male,agg_perc_fem,agg_perc_male,agg_sample,clean_n_fem,clean_n_male,clean_perc_fem,...,article_categories,article_title,authors,copyright_info,funding,publisher_id,doi,journal_title,keywords,publication_date
0,PMC9683380,"[3, 5, 6, 7, 8]","[null, null, ""\""352\"""", null, null]","[null, null, ""\""328\"""", null, null]","[null, null, null, null, null]","[null, null, null, null, null]","[""\""##3\"""", ""\""##3\"""", ""\""70\"""", ""\""##6\"""", ""\...",[352],[328],[],...,"[""Endocrinology""]",Differential diagnostic value of plain CT scan...,"[""Zhijiang Han"", ""Mengwei Wu"", ""Peiying Wei"", ...","Copyright © 2022 Han, Wu, Wei, Zhu, Zhang, Din...",[],None,10.3389/fendo.2022.1007870,Frontiers in Endocrinology,"[""adrenal gland neoplasms"", ""adrenal adenoma"",...",2022-11-09 00:00:00
1,PMC5137654,"[1, 3, 4]","[""\""30\"""", null, null]","[""\""30\"""", null, null]","[null, null, null]","[null, null, null]","[""\""60\"""", ""\""16\"""", ""\""46\""""]",[30],[30],[],...,"[""Spine""]",Posterior hemivertebra resection and monosegme...,"[""X Zhu"", ""X Wei"", ""J Chen"", ""C Li"", ""M Li"", ""...",Copyright © 2013 Royal College of Surgeons,[],650173,10.1308/003588414X13824511650173,Annals of The Royal College of Surgeons of Eng...,"[""Hemivertebra resection"", ""Monosegmental fusi...",None


In [17]:
count_data.shape[0]

166011

In [18]:
count_data.columns

Index(['pmcid', 'agg_sentence_index', 'agg_n_fem', 'agg_n_male',
       'agg_perc_fem', 'agg_perc_male', 'agg_sample', 'clean_n_fem',
       'clean_n_male', 'clean_perc_fem', 'clean_perc_male', 'clean_sample',
       'max_n_fem', 'max_n_male', 'max_perc_fem', 'max_perc_male',
       'max_sample', 'article_categories', 'article_title', 'authors',
       'copyright_info', 'funding', 'publisher_id', 'doi', 'journal_title',
       'keywords', 'publication_date'],
      dtype='object')

### Filters

In [19]:
pmc_ids_df = pd.read_csv('data/pmcids_dois_from_counts_data.txt')
pmc_ids_df.head(2)

,pmcid,doi
0,PMC9683380,10.3389/fendo.2022.1007870
1,PMC5137654,10.1308/003588414X13824511650173


In [20]:
pmc_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166011 entries, 0 to 166010
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   pmcid   166011 non-null  object
 1   doi     162158 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [21]:
pmc_ids = [x.replace('PMC', '') for x in pmc_ids_df.pmcid.unique()]
pmc_ids_df['pmcid'] = pmc_ids
print(f'Unique pmc ids: {len(pmc_ids)}')

Unique pmc ids: 166011


### Link Tables

In [22]:
# link_tables = create_df_from_csv(folders, 'link_tables')
link_tables = pd.read_csv('data/processing/link_tables.csv', index_col=0, low_memory=False)
link_tables.head()

,pmid,project_number
0,3968312,R01HL025826
1,3889259,R01HD016292
2,3918136,R01HD011011
3,3864856,T32HL007535
4,4064865,R23AM031070


In [23]:
# Convert column names to lowercase
link_tables.columns = link_tables.columns.str.lower()

In [24]:
link_tables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6437869 entries, 0 to 6437868
Data columns (total 2 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   pmid            int64 
 1   project_number  object
dtypes: int64(1), object(1)
memory usage: 147.4+ MB


In [25]:
link_tables['pmid'] = link_tables['pmid'].fillna(0).astype(str)

### Publications

In [26]:
# all_publications = create_df_from_csv(folders, 'publications')
all_publications = pd.read_csv('data/processing/publications.csv', index_col=0, low_memory=False)
all_publications.shape

(2754194, 15)

In [27]:
all_publications.head(2)

,affiliation,author_list,country,issn,journal_issue,journal_title,journal_title_abbr,journal_volume,lang,page_number,pmc_id,pmid,pub_date,pub_title,pub_year
0,NaN,"Burns, J L; Mendelman, P M; Levy, J; Stull, T ...",United States,0066-4804,1,Antimicrobial agents and chemotherapy,Antimicrob Agents Chemother,27,eng,46-54,176203,3872625,1985 Jan,A permeability barrier as a mechanism of chlor...,1985
1,NaN,"Chiou, J F; Cheng, Y C",United States,0066-4804,3,Antimicrobial agents and chemotherapy,Antimicrob Agents Chemother,27,eng,416-8,176288,2986547,1985 Mar,Interaction of Epstein-Barr virus DNA polymera...,1985


In [28]:
all_publications.columns

Index(['affiliation', 'author_list', 'country', 'issn', 'journal_issue',
       'journal_title', 'journal_title_abbr', 'journal_volume', 'lang',
       'page_number', 'pmc_id', 'pmid', 'pub_date', 'pub_title', 'pub_year'],
      dtype='object')

In [29]:
# all_publications.pub_date.unique()

In [30]:
# remove publication with only different year of publication
publications = all_publications[all_publications.duplicated(subset='pub_date') | all_publications.duplicated(subset='pub_date', keep=False)]
publications.shape

(2754194, 15)

In [31]:
print(f'Removed items: {all_publications.shape[0] - publications.shape[0]}')

Removed items: 0


In [32]:
# Convert column names to lowercase
publications.columns = publications.columns.str.lower()

In [33]:
publications.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2754194 entries, 0 to 2755775
Data columns (total 15 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   affiliation         object
 1   author_list         object
 2   country             object
 3   issn                object
 4   journal_issue       object
 5   journal_title       object
 6   journal_title_abbr  object
 7   journal_volume      object
 8   lang                object
 9   page_number         object
 10  pmc_id              int64 
 11  pmid                int64 
 12  pub_date            object
 13  pub_title           object
 14  pub_year            int64 
dtypes: int64(3), object(12)
memory usage: 336.2+ MB


In [34]:
publications['pmc_id'] = publications['pmc_id'].fillna(0).astype(int).astype(str)

In [35]:
publications['pmid'] = publications['pmid'].fillna(0).astype(int).astype(str)

### Codes

In [36]:
merged_df1 = pd.merge(pmc_ids_df, publications, left_on='pmcid', right_on='pmc_id', how='left')
print(f'Merged df: {merged_df1.shape[0]}')

Merged df: 166044


In [37]:
merged_df1.head()

,pmcid,doi,affiliation,author_list,country,issn,journal_issue,journal_title,journal_title_abbr,journal_volume,lang,page_number,pmc_id,pmid,pub_date,pub_title,pub_year
0,9683380,10.3389/fendo.2022.1007870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5137654,10.1308/003588414X13824511650173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7906844,10.5395/rde.2021.46.e7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3387267,10.1371/journal.pone.0039725,"Department of Medicine, Stem Cell Program and ...","Ma, Wenxue; Gutierrez, Alejandro; Goff, Daniel...",United States,1932-6203,6,PloS one,PLoS One,7,eng,e39725,3387267,22768113,2012,NOTCH1 signaling promotes human T-cell acute l...,2012.0
4,4221596,10.1038/mtna.2014.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
merged_df1.columns

Index(['pmcid', 'doi', 'affiliation', 'author_list', 'country', 'issn',
       'journal_issue', 'journal_title', 'journal_title_abbr',
       'journal_volume', 'lang', 'page_number', 'pmc_id', 'pmid', 'pub_date',
       'pub_title', 'pub_year'],
      dtype='object')

In [39]:
merged_df1.isnull().sum()

pmcid                      0
doi                     3853
affiliation           144344
author_list           143870
country               143952
issn                  144126
journal_issue         149226
journal_title         143865
journal_title_abbr    143865
journal_volume        143937
lang                  143865
page_number           145689
pmc_id                143865
pmid                  143865
pub_date              143865
pub_title             143865
pub_year              143865
dtype: int64

In [40]:
# Convert column names to lowercase
merged_df1.columns = merged_df1.columns.str.lower()

In [41]:
partial_df = merged_df1[(merged_df1['pmcid'].notna()) & (merged_df1['pmc_id'].notna())]
print(f'Partial df: {partial_df.shape[0]} items')

Partial df: 22179 items


In [42]:
merged_df2 = pd.merge(partial_df, link_tables, on='pmid', how='left')
merged_df2.head()

,pmcid,doi,affiliation,author_list,country,issn,journal_issue,journal_title,journal_title_abbr,journal_volume,lang,page_number,pmc_id,pmid,pub_date,pub_title,pub_year,project_number
0,3387267,10.1371/journal.pone.0039725,"Department of Medicine, Stem Cell Program and ...","Ma, Wenxue; Gutierrez, Alejandro; Goff, Daniel...",United States,1932-6203,6,PloS one,PLoS One,7,eng,e39725,3387267,22768113,2012,NOTCH1 signaling promotes human T-cell acute l...,2012.0,K08CA133103
1,3387267,10.1371/journal.pone.0039725,"Department of Medicine, Stem Cell Program and ...","Ma, Wenxue; Gutierrez, Alejandro; Goff, Daniel...",United States,1932-6203,6,PloS one,PLoS One,7,eng,e39725,3387267,22768113,2012,NOTCH1 signaling promotes human T-cell acute l...,2012.0,P01CA068484
2,7363900,10.1038/s41398-020-00916-0,"Department of Psychiatry, University of Califo...","Wierenga, Christina E; Bischoff-Grethe, Amanda...",United States,2158-3188,1,Translational psychiatry,Transl Psychiatry,10,eng,236,7363900,32669557,2020 07 16,Increased anticipatory brain response to pleas...,2020.0,R01MH113588
3,7363900,10.1038/s41398-020-00916-0,"Department of Psychiatry, University of Califo...","Wierenga, Christina E; Bischoff-Grethe, Amanda...",United States,2158-3188,1,Translational psychiatry,Transl Psychiatry,10,eng,236,7363900,32669557,2020 07 16,Increased anticipatory brain response to pleas...,2020.0,R01MH092793
4,7363900,10.1038/s41398-020-00916-0,"Department of Psychiatry, University of Califo...","Wierenga, Christina E; Bischoff-Grethe, Amanda...",United States,2158-3188,1,Translational psychiatry,Transl Psychiatry,10,eng,236,7363900,32669557,2020 07 16,Increased anticipatory brain response to pleas...,2020.0,F32MH108311


In [43]:
# Drop unuseful columns
columns_to_drop = ['journal_volume', 'lang', 'page_number', 'pmc_id']
merged_df2.drop(columns=columns_to_drop, inplace=True)

In [44]:
merged_df2.columns

Index(['pmcid', 'doi', 'affiliation', 'author_list', 'country', 'issn',
       'journal_issue', 'journal_title', 'journal_title_abbr', 'pmid',
       'pub_date', 'pub_title', 'pub_year', 'project_number'],
      dtype='object')

In [45]:
merged_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69795 entries, 0 to 69794
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pmcid               69795 non-null  object 
 1   doi                 68452 non-null  object 
 2   affiliation         68368 non-null  object 
 3   author_list         69763 non-null  object 
 4   country             69617 non-null  object 
 5   issn                69102 non-null  object 
 6   journal_issue       55134 non-null  object 
 7   journal_title       69795 non-null  object 
 8   journal_title_abbr  69795 non-null  object 
 9   pmid                69795 non-null  object 
 10  pub_date            69795 non-null  object 
 11  pub_title           69795 non-null  object 
 12  pub_year            69795 non-null  float64
 13  project_number      69795 non-null  object 
dtypes: float64(1), object(13)
memory usage: 7.5+ MB


In [46]:
merged_df2.head(2)

,pmcid,doi,affiliation,author_list,country,issn,journal_issue,journal_title,journal_title_abbr,pmid,pub_date,pub_title,pub_year,project_number
0,3387267,10.1371/journal.pone.0039725,"Department of Medicine, Stem Cell Program and ...","Ma, Wenxue; Gutierrez, Alejandro; Goff, Daniel...",United States,1932-6203,6,PloS one,PLoS One,22768113,2012,NOTCH1 signaling promotes human T-cell acute l...,2012.0,K08CA133103
1,3387267,10.1371/journal.pone.0039725,"Department of Medicine, Stem Cell Program and ...","Ma, Wenxue; Gutierrez, Alejandro; Goff, Daniel...",United States,1932-6203,6,PloS one,PLoS One,22768113,2012,NOTCH1 signaling promotes human T-cell acute l...,2012.0,P01CA068484


In [47]:
# Dataframe fro the merge - convert to string
merged_df2.fillna('', inplace = True)
merged_df2 = merged_df2.astype(str)

In [48]:
# Drop duplicates
filter_df2 = merged_df2.drop_duplicates(keep='last')
print(f'Removed items: {merged_df2.shape[0] - filter_df2.shape[0]}')

Removed items: 226


In [49]:
filter_df = filter_df2[['pmcid', 'doi', 'pmid', 'project_number']]
filter_df.head()

,pmcid,doi,pmid,project_number
0,3387267,10.1371/journal.pone.0039725,22768113,K08CA133103
1,3387267,10.1371/journal.pone.0039725,22768113,P01CA068484
2,7363900,10.1038/s41398-020-00916-0,32669557,R01MH113588
3,7363900,10.1038/s41398-020-00916-0,32669557,R01MH092793
4,7363900,10.1038/s41398-020-00916-0,32669557,F32MH108311


In [50]:
# Drop duplicates
codes_df = filter_df.drop_duplicates(keep='last')
print(f'Removed items: {filter_df.shape[0] - codes_df.shape[0]}')

Removed items: 70


In [51]:
print(f"Unique pmc-ids: {len(codes_df.pmcid.unique())}")

Unique pmc-ids: 22146


### Projects

In [52]:
projects_number = list(codes_df.project_number.unique())
print(f'Unique project codes: {len(projects_number)}')

Unique project codes: 33200


In [53]:
# Check if a list contains Nan, nulls or None
check_list_integrity(projects_number)

None: False
Nan: False
Null: False


In [54]:
all_projects = create_df_from_csv(folders, 'projects', projects_number)  # filter: pmc_ids
# all_projects = pd.read_csv('data/processing/projects.csv', index_col=0, low_memory=False)
all_projects.shape

Number of projects: 616345


(616345, 47)

In [55]:
all_projects.columns = all_projects.columns.str.lower()

In [56]:
all_projects.columns

Index(['application_id', 'activity', 'administering_ic', 'application_type',
       'arra_funded', 'award_notice_date', 'budget_start', 'budget_end',
       'cfda_code', 'core_project_num', 'ed_inst_type', 'foa_number',
       'full_project_num', 'subproject_id', 'funding_ics', 'fy', 'ic_name',
       'nih_spending_cats', 'org_city', 'org_country', 'org_dept',
       'org_district', 'org_duns', 'org_fips', 'org_name', 'org_state',
       'org_zipcode', 'phr', 'pi_ids', 'pi_names', 'program_officer_name',
       'project_start', 'project_end', 'project_terms', 'project_title',
       'serial_number', 'study_section', 'study_section_name', 'suffix',
       'support_year', 'total_cost', 'total_cost_sub_project',
       'funding_mechanism', 'direct_cost_amt', 'indirect_cost_amt',
       'org_ipf_code', 'opportunity number'],
      dtype='object')

In [57]:
int_conversion =['application_type', 'support_year', 'total_cost',
                 'total_cost_sub_project', 'direct_cost_amt', 'indirect_cost_amt', 'org_ipf_code']

In [58]:
for i in int_conversion:
    all_projects.fillna(0, inplace = True)
    all_projects[i] = all_projects[i].astype(int)
    all_projects[i] = all_projects[i].replace(0, '')

In [59]:
# Drop unuseful columns
columns_to_drop = ['cfda_code', 'org_district', 'org_duns', 'org_zipcode', 'pi_ids', 'suffix']
if 'cfda_code' in all_projects.columns:
    all_projects.drop(columns=columns_to_drop, inplace=True)

In [60]:
count_duplicates(all_projects)

Duplicate Rows: 0


In [61]:
repeated_values =count_repeat_values(all_projects, 'core_project_num')

Values that repeat multiple times: 31117


In [62]:
rv = all_projects[all_projects.core_project_num == repeated_values[0]]
print(f'One project name > {rv.shape[0]} rows')

One project name > 4915 rows


In [63]:
# Drop duplicates
filter_projects = all_projects.drop_duplicates(keep='last')
print(f'Removed items: {all_projects.shape[0] - filter_projects.shape[0]}')

Removed items: 0


In [64]:
projects = filter_projects[filter_projects["core_project_num"].isin(projects_number)]
projects.shape

(616345, 41)

In [65]:
print(f'Removed items: {all_projects.shape[0] - projects.shape[0]}')

Removed items: 0


In [66]:
projects.columns = projects.columns.str.lower()

In [67]:
projects.columns

Index(['application_id', 'activity', 'administering_ic', 'application_type',
       'arra_funded', 'award_notice_date', 'budget_start', 'budget_end',
       'core_project_num', 'ed_inst_type', 'foa_number', 'full_project_num',
       'subproject_id', 'funding_ics', 'fy', 'ic_name', 'nih_spending_cats',
       'org_city', 'org_country', 'org_dept', 'org_fips', 'org_name',
       'org_state', 'phr', 'pi_names', 'program_officer_name', 'project_start',
       'project_end', 'project_terms', 'project_title', 'serial_number',
       'study_section', 'study_section_name', 'support_year', 'total_cost',
       'total_cost_sub_project', 'funding_mechanism', 'direct_cost_amt',
       'indirect_cost_amt', 'org_ipf_code', 'opportunity number'],
      dtype='object')

In [68]:
# Dataframe fro the merge - convert to string
projects.fillna('', inplace = True)
projects = projects.astype(str)

### Save partial results

In [ ]:
pmc_ids_df.to_csv('data/processing/codes.csv')
publications.to_csv('data/processing/publications.csv')
link_tables.to_csv('data/processing/link_tables.csv')
merged_df2.to_csv('data/processing/merge_df2.csv')
projects.to_csv('data/processing/projects.csv')